In [1]:
# magic! (don't worry about this)
%load_ext autoreload
%autoreload 2
#%matplotlib inline

In [2]:
# let us import some useful things
from lib import *
from classifiers import *
#from graphs import *
from sklearn import tree


In [ ]:
# load the data same as before
microarray_file_name = '../data/leukemia_ALL_AML_matrix.txt'
labels_file_name = '../data/leukemia_ALL_AML_labels.txt'
data_store = DataSet(microarray_file_name, labels_file_name)